Initialization

In [3]:
import pandas as pd
import requests
import datetime

def fetch_race_results(year, round_number):
    url = f"https://ergast.com/api/f1/{year}/{round_number}/results.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def process_race_results(results, year, round_number):
    race_data = []
    for race in results['MRData']['RaceTable']['Races']:
        race_name = race['raceName']
        race_date = race['date']
        circuit = race['Circuit']['circuitName']
        for result in race['Results']:
            position = result['position']
            driver = result['Driver']
            driver_name = f"{driver['givenName']} {driver['familyName']}"
            constructor = result['Constructor']['name']
            grid = result['grid']
            laps = result['laps']
            status = result['status']
            race_data.append({
                'Year': year,
                'Round': round_number,
                'Race': race_name,
                'Date': race_date,
                'Circuit': circuit,
                'Position': position,
                'Driver': driver_name,
                'Constructor': constructor,
                'Grid': grid,
                'Laps': laps,
                'Status': status
            })
    return race_data

def fetch_f1_data(start_year, end_year):
    all_race_results = []
    for year in range(start_year, end_year + 1):
        print(f"Fetching data for {year}...")
        round_number = 1
        while True:
            results = fetch_race_results(year, round_number)
            if results and results['MRData']['RaceTable']['Races']:
                race_data = process_race_results(results, year, round_number)
                all_race_results.extend(race_data)
                round_number += 1
            else:
                break
    return all_race_results

# Fetch data from 2017 to the present year
current_year = datetime.datetime.now().year
race_results = fetch_f1_data(2017, current_year)

# Convert the race results to a DataFrame
df_race_results = pd.DataFrame(race_results)

# Display the first few rows of the DataFrame
df_race_results.head()

# Save the DataFrame to a CSV file
df_race_results.to_csv('f1_race_results_2017_present.csv', index=False)


Fetching data for 2017...
Fetching data for 2018...
Fetching data for 2019...
Fetching data for 2020...
Fetching data for 2021...
Fetching data for 2022...
Fetching data for 2023...
Fetching data for 2024...


In [15]:
def fetch_sprint_race_results(year, round_number):
    url = f"https://ergast.com/api/f1/{year}/{round_number}/sprint.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def process_sprint_race_results(results, year, round_number):
    sprint_race_data = []
    for race in results['MRData']['RaceTable']['Races']:
        race_name = race['raceName']
        race_date = race['date']
        circuit = race['Circuit']['circuitName']
        for result in race['SprintResults']:
            position = result['position']
            driver = result['Driver']
            driver_name = f"{driver['givenName']} {driver['familyName']}"
            constructor = result['Constructor']['name']
            grid = result['grid']
            laps = result['laps']
            status = result['status']
            sprint_race_data.append({
                'Year': year,
                'Round': round_number,
                'Race': race_name,
                'Date': race_date,
                'Circuit': circuit,
                'Position': position,
                'Driver': driver_name,
                'Constructor': constructor,
                'Grid': grid,
                'Laps': laps,
                'Status': status,
                'RaceType': 'Sprint'
            })
    return sprint_race_data

def fetch_f1_sprint_data(start_year, end_year):
    all_sprint_race_results = []
    for year in range(start_year, end_year + 1):
        print(f"Fetching sprint race data for {year}...")
        round_number = 1
        while True:
            sprint_results = fetch_sprint_race_results(year, round_number)
            if sprint_results and sprint_results['MRData']['RaceTable']['Races']:
                sprint_race_data = process_sprint_race_results(sprint_results, year, round_number)
                all_sprint_race_results.extend(sprint_race_data)
                round_number += 1
            else:
                break
    return all_sprint_race_results

# Fetch sprint race data from 2017 to the present year
sprint_race_results = fetch_f1_sprint_data(2017, current_year)

# Convert the sprint race results to a DataFrame
df_sprint_results = pd.DataFrame(sprint_race_results)

# Display the first few rows of the DataFrame
df_sprint_results.head()

# Save the DataFrame to a CSV file
df_sprint_results.to_csv('f1_sprint_race_results_2017_present.csv', index=False)


Fetching sprint race data for 2017...
Fetching sprint race data for 2018...
Fetching sprint race data for 2019...
Fetching sprint race data for 2020...
Fetching sprint race data for 2021...
Fetching sprint race data for 2022...
Fetching sprint race data for 2023...
Fetching sprint race data for 2024...


In [31]:
import pandas as pd
import requests
import datetime

def fetch_season_results(year):
    url = f"https://ergast.com/api/f1/{year}/results.json?limit=1000"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def fetch_sprint_results(year):
    url = f"https://ergast.com/api/f1/{year}/sprint.json?limit=1000"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def process_results(results, year, race_type):
    race_data = []
    for race in results.get('MRData', {}).get('RaceTable', {}).get('Races', []):
        race_info = {
            'Year': year,
            'Round': race.get('round', 'N/A'),
            'RaceName': race.get('raceName', 'Unknown Race'),
            'Date': race.get('date', 'Unknown Date'),
            'Time': race.get('time', 'N/A'),
            'CircuitId': race.get('Circuit', {}).get('circuitId', 'Unknown Circuit'),
            'CircuitName': race.get('Circuit', {}).get('circuitName', 'Unknown Circuit'),
            'LocationLat': race.get('Circuit', {}).get('Location', {}).get('lat', 'N/A'),
            'LocationLong': race.get('Circuit', {}).get('Location', {}).get('long', 'N/A'),
            'Locality': race.get('Circuit', {}).get('Location', {}).get('locality', 'N/A'),
            'Country': race.get('Circuit', {}).get('Location', {}).get('country', 'N/A'),
            'RaceType': race_type
        }
        if race_type == 'Race':
            results_key = 'Results'
        else:
            results_key = 'SprintResults'

        for result in race.get(results_key, []):
            driver = result.get('Driver', {})
            constructor = result.get('Constructor', {})
            time_info = result.get('Time', {})
            fastest_lap = result.get('FastestLap', {}).get('Time', {})
            race_data.append({
                **race_info,
                'Position': result.get('position', 'N/A'),
                'PositionText': result.get('positionText', 'N/A'),
                'Points': result.get('points', 'N/A'),
                'DriverId': driver.get('driverId', 'N/A'),
                'DriverCode': driver.get('code', 'N/A'),
                'DriverName': f"{driver.get('givenName', 'Unknown')} {driver.get('familyName', 'Unknown')}",
                'DriverDOB': driver.get('dateOfBirth', 'N/A'),
                'DriverNationality': driver.get('nationality', 'N/A'),
                'ConstructorId': constructor.get('constructorId', 'N/A'),
                'ConstructorName': constructor.get('name', 'N/A'),
                'ConstructorNationality': constructor.get('nationality', 'N/A'),
                'Grid': result.get('grid', 'N/A'),
                'Laps': result.get('laps', 'N/A'),
                'Status': result.get('status', 'N/A'),
                'Time': time_info.get('time', 'N/A'),
                'Millis': time_info.get('millis', 'N/A'),
                'FastestLapTime': fastest_lap.get('time', 'N/A')
            })
    return race_data

def fetch_f1_data(start_year, end_year):
    all_race_results = []
    for year in range(start_year, end_year + 1):
        print(f"Fetching race data for {year}...")
        # Fetch entire season race results
        results = fetch_season_results(year)
        if results and results.get('MRData', {}).get('RaceTable', {}).get('Races', []):
            race_data = process_results(results, year, 'Race')
            all_race_results.extend(race_data)
            print(f"Fetched race results for {year}.")

        # Fetch entire season sprint results if year is 2021 or later
        if year >= 2021:
            sprint_results = fetch_sprint_results(year)
            if sprint_results and sprint_results.get('MRData', {}).get('RaceTable', {}).get('Races', []):
                sprint_race_data = process_results(sprint_results, year, 'Sprint')
                all_race_results.extend(sprint_race_data)
                print(f"Fetched sprint race results for {year}.")

    return all_race_results

# Fetch data from 2017 to the present year
current_year = datetime.datetime.now().year
race_results = fetch_f1_data(2017, current_year)

# Convert the race results to a DataFrame
df_race_results = pd.DataFrame(race_results)

# Display the first few rows of the DataFrame
display(df_race_results.head(10))

# Save the DataFrame to a CSV file
# df_race_results.to_csv('f1_race_results_2017_present.csv', index=False)


,Year,Round,RaceName,Date,Time,CircuitId,CircuitName,LocationLat,LocationLong,Locality,...,DriverDOB,DriverNationality,ConstructorId,ConstructorName,ConstructorNationality,Grid,Laps,Status,Millis,FastestLapTime
0,2017,1,Australian Grand Prix,2017-03-26,1:24:11.672,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1987-07-03,German,ferrari,Ferrari,Italian,2,57,Finished,5051672,1:26.638
1,2017,1,Australian Grand Prix,2017-03-26,+9.975,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1985-01-07,British,mercedes,Mercedes,German,1,57,Finished,5061647,1:27.033
2,2017,1,Australian Grand Prix,2017-03-26,+11.250,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1989-08-28,Finnish,mercedes,Mercedes,German,3,57,Finished,5062922,1:26.593
3,2017,1,Australian Grand Prix,2017-03-26,+22.393,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1979-10-17,Finnish,ferrari,Ferrari,Italian,4,57,Finished,5074065,1:26.538
4,2017,1,Australian Grand Prix,2017-03-26,+28.827,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1997-09-30,Dutch,red_bull,Red Bull,Austrian,5,57,Finished,5080499,1:26.964
5,2017,1,Australian Grand Prix,2017-03-26,+1:23.386,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1981-04-25,Brazilian,williams,Williams,British,7,57,Finished,5135058,1:28.045
6,2017,1,Australian Grand Prix,2017-03-26,N/A,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1990-01-26,Mexican,force_india,Force India,Indian,10,56,+1 Lap,N/A,1:28.336
7,2017,1,Australian Grand Prix,2017-03-26,N/A,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1994-09-01,Spanish,toro_rosso,Toro Rosso,Italian,8,56,+1 Lap,N/A,1:27.677
8,2017,1,Australian Grand Prix,2017-03-26,N/A,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1994-04-26,Russian,toro_rosso,Toro Rosso,Italian,9,56,+1 Lap,N/A,1:26.711
9,2017,1,Australian Grand Prix,2017-03-26,N/A,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,...,1996-09-17,French,force_india,Force India,Indian,13,56,+1 Lap,N/A,1:28.475


In [32]:
import pandas as pd
import requests
import datetime

def fetch_season_results(year, result_type='results'):
    url = f"https://ergast.com/api/f1/{year}/{result_type}.json?limit=1000"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def process_results(results, year, result_type):
    race_data = []
    for race in results.get('MRData', {}).get('RaceTable', {}).get('Races', []):
        race_info = {
            'Year': year,
            'Round': race.get('round', 'N/A'),
            'RaceName': race.get('raceName', 'Unknown Race'),
            'Date': race.get('date', 'Unknown Date'),
            'Time': race.get('time', 'N/A'),
            'CircuitId': race.get('Circuit', {}).get('circuitId', 'Unknown Circuit'),
            'CircuitName': race.get('Circuit', {}).get('circuitName', 'Unknown Circuit'),
            'LocationLat': race.get('Circuit', {}).get('Location', {}).get('lat', 'N/A'),
            'LocationLong': race.get('Circuit', {}).get('Location', {}).get('long', 'N/A'),
            'Locality': race.get('Circuit', {}).get('Location', {}).get('locality', 'N/A'),
            'Country': race.get('Circuit', {}).get('Location', {}).get('country', 'N/A'),
            'ResultType': result_type
        }

        if result_type == 'results':
            results_key = 'Results'
        elif result_type == 'sprint':
            results_key = 'SprintResults'
        elif result_type == 'qualifying':
            results_key = 'QualifyingResults'

        for result in race.get(results_key, []):
            driver = result.get('Driver', {})
            constructor = result.get('Constructor', {})
            time_info = result.get('Time', {})
            fastest_lap = result.get('FastestLap', {}).get('Time', {})
            qualifying_times = {
                'Q1': result.get('Q1', 'N/A'),
                'Q2': result.get('Q2', 'N/A'),
                'Q3': result.get('Q3', 'N/A')
            }
            race_data.append({
                **race_info,
                'Position': result.get('position', 'N/A'),
                'PositionText': result.get('positionText', 'N/A'),
                'Points': result.get('points', 'N/A'),
                'DriverId': driver.get('driverId', 'N/A'),
                'DriverCode': driver.get('code', 'N/A'),
                'DriverName': f"{driver.get('givenName', 'Unknown')} {driver.get('familyName', 'Unknown')}",
                'DriverDOB': driver.get('dateOfBirth', 'N/A'),
                'DriverNationality': driver.get('nationality', 'N/A'),
                'ConstructorId': constructor.get('constructorId', 'N/A'),
                'ConstructorName': constructor.get('name', 'N/A'),
                'ConstructorNationality': constructor.get('nationality', 'N/A'),
                'Grid': result.get('grid', 'N/A'),
                'Laps': result.get('laps', 'N/A'),
                'Status': result.get('status', 'N/A'),
                'Time': time_info.get('time', 'N/A'),
                'Millis': time_info.get('millis', 'N/A'),
                'FastestLapTime': fastest_lap.get('time', 'N/A'),
                **qualifying_times
            })
    return race_data

def fetch_f1_data(start_year, end_year):
    all_race_results = []
    for year in range(start_year, end_year + 1):
        print(f"Fetching race data for {year}...")

        # Fetch entire season race results
        race_results = fetch_season_results(year, 'results')
        if race_results and race_results.get('MRData', {}).get('RaceTable', {}).get('Races', []):
            race_data = process_results(race_results, year, 'results')
            all_race_results.extend(race_data)
            print(f"Fetched race results for {year}.")

        # Fetch entire season sprint results if year is 2021 or later
        if year >= 2021:
            sprint_results = fetch_season_results(year, 'sprint')
            if sprint_results and sprint_results.get('MRData', {}).get('RaceTable', {}).get('Races', []):
                sprint_race_data = process_results(sprint_results, year, 'sprint')
                all_race_results.extend(sprint_race_data)
                print(f"Fetched sprint race results for {year}.")

        # Fetch entire season qualifying results
        qualifying_results = fetch_season_results(year, 'qualifying')
        if qualifying_results and qualifying_results.get('MRData', {}).get('RaceTable', {}).get('Races', []):
            qualifying_data = process_results(qualifying_results, year, 'qualifying')
            all_race_results.extend(qualifying_data)
            print(f"Fetched qualifying results for {year}.")

    return all_race_results

# Fetch data from 2017 to the present year
current_year = datetime.datetime.now().year
race_results = fetch_f1_data(2017, current_year)

# Convert the race results to a DataFrame
df_race_results = pd.DataFrame(race_results)

# Display the first few rows of the DataFrame
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)
print(df_race_results.head(100))

# Save the DataFrame to a CSV file
df_race_results.to_csv('f1_race_results_2017_present.csv', index=False)


Fetching race data for 2017...
Fetched race results for 2017.
Fetched qualifying results for 2017.
Fetching race data for 2018...
Fetched race results for 2018.
Fetched qualifying results for 2018.
Fetching race data for 2019...
Fetched race results for 2019.
Fetched qualifying results for 2019.
Fetching race data for 2020...
Fetched race results for 2020.
Fetched qualifying results for 2020.
Fetching race data for 2021...
Fetched race results for 2021.
Fetched sprint race results for 2021.
Fetched qualifying results for 2021.
Fetching race data for 2022...
Fetched race results for 2022.
Fetched sprint race results for 2022.
Fetched qualifying results for 2022.
Fetching race data for 2023...
Fetched race results for 2023.
Fetched sprint race results for 2023.
Fetched qualifying results for 2023.
Fetching race data for 2024...
Fetched race results for 2024.
Fetched sprint race results for 2024.
Fetched qualifying results for 2024.
    Year Round               RaceName        Date       

In [34]:
import pandas as pd

# Display all columns in the DataFrame
print("Columns in DataFrame:")
print(df_race_results.columns)

# Display unique values for each column
for column in df_race_results.columns:
    unique_values = df_race_results[column].unique()
    print(f"\nColumn: {column}")
    print(f"Unique values ({len(unique_values)}): {unique_values[:10]}")  # Display first 10 unique values

# Save the unique values summary to a CSV file for further inspection
summary = {column: df_race_results[column].unique() for column in df_race_results.columns}
df_summary = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in summary.items()]))



Columns in DataFrame:
Index(['Year', 'Round', 'RaceName', 'Date', 'Time', 'CircuitId', 'CircuitName',
       'LocationLat', 'LocationLong', 'Locality', 'Country', 'ResultType',
       'Position', 'PositionText', 'Points', 'DriverId', 'DriverCode',
       'DriverName', 'DriverDOB', 'DriverNationality', 'ConstructorId',
       'ConstructorName', 'ConstructorNationality', 'Grid', 'Laps', 'Status',
       'Millis', 'FastestLapTime', 'Q1', 'Q2', 'Q3', 'DriverQualifyingPoints'],
      dtype='object')

Column: Year
Unique values (8): [2017 2018 2019 2020 2021 2022 2023 2024]

Column: Round
Unique values (22): ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10']

Column: RaceName
Unique values (37): ['Australian Grand Prix' 'Chinese Grand Prix' 'Bahrain Grand Prix'
 'Russian Grand Prix' 'Spanish Grand Prix' 'Monaco Grand Prix'
 'Canadian Grand Prix' 'Azerbaijan Grand Prix' 'Austrian Grand Prix'
 'British Grand Prix']

Column: Date
Unique values (154): ['2017-03-26' '2017-04-09' '2017-04-16' '2017-04-30'

In [35]:
import pandas as pd

# Function to add and calculate missing columns
def add_and_calculate_missing_columns(df):
    # Calculate PositionsGained and PositionsLost based on Grid and Position
    df['PositionsGained'] = df['Grid'].astype(int) - df['Position'].astype(int)
    df['PositionsLost'] = -df['PositionsGained']

    # Use PositionsGained as a proxy for Overtakes
    df['Overtakes'] = df['PositionsGained']

    # Calculate FastestLap based on the fastest lap time among all drivers in each race
    df['FastestLap'] = df.groupby(['Year', 'Round'])['FastestLapTime'].transform(lambda x: x == x.min())

    # Assuming DriverOfTheDay data needs to be initialized for simplicity
    df['DriverOfTheDay'] = False  # This should be updated with actual Driver of the Day data if available

    # Assuming FastestPitstop data needs to be initialized for simplicity
    df['FastestPitstop'] = 0  # This should be updated with actual fastest pitstop data if available

    return df

# Add and calculate missing columns in the DataFrame
df_race_results = add_and_calculate_missing_columns(df_race_results)

# Define non-finishing statuses
non_finishing_statuses = [
    'Vibrations', 'Suspension', 'Brakes', 'Power Unit', 'Hydraulics', 'Water leak',
    'Driveshaft', 'Accident', 'Collision', 'Engine', 'Gearbox', 'Electrical',
    'Puncture', 'Fuel pressure', 'Retired', 'Clutch', 'Transmission', 'Oil leak',
    'Radiator', 'Wheel', 'Tyre', 'Heat Shield', 'Exhaust', 'Turbo', 'Steering',
    'Fuel pump', 'Ignition', 'Wheel nut', 'Technical', 'Withdrew', 'Spun off',
    'Handling', 'Chassis', 'Differential', 'Overheating', 'Fuel system',
    'Rear wing', 'Throttle', 'Alternator', 'Battery', 'Water pump', 'Fire',
    'Wheel bearing', 'Blocked', 'Not classified', 'DNF'
]

# Qualifying points calculation
def calculate_driver_qualifying_points(position, status):
    if status == 'Disqualified':
        return -15
    elif status == 'No time set':
        return -5

    points = {1: 10, 2: 9, 3: 8, 4: 7, 5: 6, 6: 5, 7: 4, 8: 3, 9: 2, 10: 1}
    return points.get(position, 0)

def calculate_constructor_qualifying_points(df_qualifying, constructor):
    constructor_data = df_qualifying[df_qualifying['ConstructorName'] == constructor]
    total_points = constructor_data['DriverQualifyingPoints'].sum()

    q2_count = (constructor_data['Position'].astype(int) <= 15).sum()
    q3_count = (constructor_data['Position'].astype(int) <= 10).sum()

    if q2_count == 0:
        total_points -= 1
    elif q2_count == 1:
        total_points += 1
    else:
        total_points += 3

    if q3_count == 1:
        total_points += 5
    elif q3_count == 2:
        total_points += 10

    return total_points

# Sprint points calculation
def calculate_driver_sprint_points(row):
    points = {1: 8, 2: 7, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1}
    base_points = points.get(row['Position'], 0)

    if row['Status'] == 'Disqualified':
        return base_points - 25
    elif row['Status'] in non_finishing_statuses:
        return base_points - 20

    return base_points + row['PositionsGained'] + row['Overtakes'] + (5 if row['FastestLap'] else 0)

# Race points calculation
def calculate_driver_race_points(row):
    points = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}
    base_points = points.get(row['Position'], 0)

    if row['Status'] == 'Disqualified':
        return base_points - 25
    elif row['Status'] in non_finishing_statuses:
        return base_points - 20

    return base_points + row['PositionsGained'] + row['Overtakes'] + (10 if row['FastestLap'] else 0) + (10 if row['DriverOfTheDay'] else 0)

# Constructor points calculation for sprint and race
def calculate_constructor_points(df, constructor, result_type, driver_points_col):
    constructor_data = df[(df['ConstructorName'] == constructor) & (df['ResultType'] == result_type)]
    total_points = constructor_data[driver_points_col].sum()

    if result_type == 'results':
        fastest_pitstops = constructor_data['FastestPitstop'].sum()
        total_points += min(fastest_pitstops * 10, 18)  # Max 18 points for fastest pitstops

    return total_points

def score_fantasy_points(df):
    # Calculate qualifying points
    df['DriverQualifyingPoints'] = df.apply(lambda row: calculate_driver_qualifying_points(int(row['Position']), row['Status']) if row['ResultType'] == 'qualifying' else 0, axis=1)

    # Calculate sprint points
    df['DriverSprintPoints'] = df.apply(lambda row: calculate_driver_sprint_points(row) if row['ResultType'] == 'sprint' else 0, axis=1)

    # Calculate race points
    df['DriverRacePoints'] = df.apply(lambda row: calculate_driver_race_points(row) if row['ResultType'] == 'results' else 0, axis=1)

    # Calculate constructor points
    constructors = df['ConstructorName'].unique()
    for constructor in constructors:
        qualifying_points = calculate_constructor_qualifying_points(df[df['ResultType'] == 'qualifying'], constructor)
        sprint_points = calculate_constructor_points(df, constructor, 'sprint', 'DriverSprintPoints')
        race_points = calculate_constructor_points(df, constructor, 'results', 'DriverRacePoints')

        df.loc[df['ConstructorName'] == constructor, 'ConstructorQualifyingPoints'] = qualifying_points
        df.loc[df['ConstructorName'] == constructor, 'ConstructorSprintPoints'] = sprint_points
        df.loc[df['ConstructorName'] == constructor, 'ConstructorRacePoints'] = race_points

    return df

# Example usage
df_race_results = score_fantasy_points(df_race_results)

# Display the first few rows of the DataFrame
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)
print(df_race_results.head(100))

# Save the DataFrame to a CSV file
# df_race_results.to_csv('f1_fantasy_points_2017_present.csv', index=False)


ValueError: invalid literal for int() with base 10: 'N/A'

In [29]:
# Filter the DataFrame for 2024 races
df_race_2024 = df_race_results[df_race_results['Year'] == 2024]

# Group by the race name and count the occurrences
race_counts = df_race_2024['RaceType'].value_counts()

# Display the results
print(race_counts)


RaceType
Race      179
Sprint     40
Name: count, dtype: int64


In [16]:
# Display the first few rows of the DataFrame
df_sprint_results.tail(30)

# Save the DataFrame to a CSV file
# df_race_results.to_csv('f1_race_results_2017_present.csv', index=False)


""
